# Automatic differentiation

$F: \mathbb{R}^n \rightarrow \mathbb{R}$ is a scalar valued function $F$ that takes in a vector input of dimensionality $n$, and returns a scalar.

Can also write as $F(\mathbf{x}\in\mathbb{R}^n) \rightarrow y \in \mathbb{R}$.

If we could break down $F$ into a composition of e.g. four other functions, we would write that as 
$F = D \circ C \circ B \circ A$, which is equivalent to $y = F(\mathbf{x}) = D(C(B(A(\mathbf{x}))))$, where we can interpret the computation as starting from the inner level, i.e. application of $A$ to $\mathbf{x}$, then $B$ to $\mathbf{a} =$ the output of $A(\mathbf{x})$), and so on. Let's also define $\mathbf{b} = B(\mathbf{a})$, $\mathbf{c} = C(\mathbf{b})$, and $y = D(\mathbf{c})$.

Function composition is important to cover, since a core idea of autodiff is to break down the gradient of a whole into the composition of the gradient of its parts via the chain rule.

Recall the **Jacobian matrix**:

$$
\mathbf{J}(F) = F'(\mathbf{x}) =  \left[ \frac{\partial y}{\partial x_1} , \frac{\partial y}{\partial x_2}, \cdots, \frac{\partial y}{\partial x_n}\right]
$$

For our example, we can break this down into a product of individual Jacobian matricies for each of the intermediate functions:



One can see the sizes of each of the intermediate matricies:

- `size($D'(\mathbf{c})$) = (1,len(c))`
- `size($C'(\mathbf{b})$) = (len(c), len(b))`
- `size($B'(\mathbf{a})$) = (len(b), len(a))`
- `size($A'(\mathbf{x})$) = (len(a), n)` (horizontal size on the last image is n)

The size of the final Jacobian matrix is then  `(1, n)`, as shown earlier.

This kind of sequential matrix multiplication can be called "accumulating" the Jacobian piece-by-piece. The order of the multiplication of these matricies (i.e. where to put the parentheses) can matter to optimise computational load, but two cases are of particular interest: 

- **Forward accumulation**: ****Start from the input and work forwards to the output (right to left)

$$
F'(\mathbf{x}) = \frac{\partial y}{\partial \mathbf{c}}\left(\frac{\partial \mathbf{c}}{\partial \mathbf{b}}\left(\frac{\partial \mathbf{b}}{\partial \mathbf{a}} \cdot \frac{\partial \mathbf{a}}{\partial \mathbf{x}}\right)\right),
$$

$$
\frac{\partial \mathbf{b}}{\partial \mathbf{a}} \cdot \frac{\partial \mathbf{a}}{\partial \mathbf{x}}\ =\frac{\partial \mathbf{b}}{\partial \mathbf{x}}=\left[\begin{array}{ccc}\frac{\partial b_{1}}{\partial x_{1}} & \cdots & \frac{\partial b_{1}}{\partial x_{n}} \\\vdots & \ddots & \vdots \\\frac{\partial b_{m}}{\partial x_{1}} & \cdots & \frac{\partial b_{m}}{\partial x_{n}}\end{array}\right].
$$

- **Reverse accumulation**: vice-versa!

$$
F'(\mathbf{x}) = \left( \left( \frac{\partial y}{\partial \mathbf{c}} \cdot \frac{\partial \mathbf{c}}{\partial \mathbf{b}} \right)\frac{\partial \mathbf{b}}{\partial \mathbf{a}} \right)\frac{\partial \mathbf{a}}{\partial \mathbf{x}},
$$

 

$$
\frac{\partial y}{\partial \mathbf{c}} \cdot \frac{\partial \mathbf{c}}{\partial \mathbf{b}}\ =\frac{\partial y}{\partial \mathbf{b}}= \left[ \frac{\partial y}{\partial b_1} , \frac{\partial y}{\partial b_2}, \cdots, \frac{\partial y}{\partial b_n}\right]
$$

Note that the size of every intermediate matrix product is always `(1, -)`, meaning if we have this situation where we're going from $\mathbb{R}^n \rightarrow \mathbb{R}$, reverse accumulation is much more efficient in terms of memory usage and compute, since we're only ever considering vectors and not matricies. This is the typical setting with a neural network: we have a very large input space (parameters), and we want to evaluate the Jacobian matrix of a scalar (loss function) with respect to those parameters.

If we had the complementary setting, i.e. $\mathbb{R} \rightarrow \mathbb{R}^n$, we would probably want to compute the Jacobian with forward accumulation instead, and apply the logic above in reverse.

### Jacobian-vector/vector-Jacobian products (JVP/VJPs)

Say we wanted to compute the product between the Jacobian of $F$ and some vector $v$, which we'll abbreviate to JVP (Jacobian-vector product). Despite the purpose of this query being not obvious at first — let alone warranting it a dedicated nickname —  it will prove to be useful in just a few lines, so hang tight.

We can write this query with the same operation ordering as with the *forward* accumulation of a Jacobian:

$$
F'(\mathbf{x})\,\mathbf{v} = \frac{\partial y}{\partial \mathbf{c}}\left(\frac{\partial \mathbf{c}}{\partial \mathbf{b}}\left(\frac{\partial \mathbf{b}}{\partial \mathbf{a}} \left(\frac{\partial \mathbf{a}}{\partial \mathbf{x}} \cdot \mathbf{v} \right) \right)\right)
$$

Thinking of the rules of matrix multiplication, we note that $\frac{\partial \mathbf{a}}{\partial \mathbf{x}} \cdot \mathbf{v}$ is only tractable if $\mathbf{v}$ is of size `(n, 1)`, since $\frac{\partial \mathbf{a}}{\partial \mathbf{x}}$  is of size `(len(a), n)`. Provided this is the case, all following computations will include 1 as one of the outer dimensions, meaning we once again only need to consider intermediary vectors instead of matricies when computing a

What does it mean to "differentiate through" something?